# This notebook will serve as a way to implement character generation LSTM and other implementation

In [218]:
import numpy as np
import pandas as pd
import torch
import pickle
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, CharLSTM

In [219]:
with open(r"..\Corpus\Encoding_RNN_LSTM\Char_level\encoding_map.pkl", "rb") as f:
    mapping = pickle.load(f)

int2char = {i: ch for ch, i in mapping.items()}
nb_char = len(int2char)

In [220]:
matrix = pd.read_csv(r"..\Markov\transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [221]:
embedding_dim = 64
hidden_size = 384
vocab_size = len(mapping)

model = CharLSTM(vocab_size, embedding_dim, hidden_size, num_layers=2)   

# Teacher forcing = 1 during training

In [222]:
ckpt = torch.load("..\Models\LSTM_model_full_forced.pt", map_location="cpu")
model.load_state_dict(ckpt["model_state_dict"])

<All keys matched successfully>

I will give the same structure for RNN and LSTM to compare

In [223]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

['<BEGINNING>', '<COUPLET>', '<PONT>', '<COUPLET>', '<PONT>', '<COUPLET>', '<REFRAIN>', '<END>']


Forbidden to generate

In [224]:
forbidden_generation = [i for i in range(64,72)]
[int2char[i] for i in forbidden_generation]

['α', 'β', 'γ', 'ε', 'ζ', 'η', 'θ', '€']

In [225]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [226]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1, device="cpu")

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoi est contre le plus blanc, j'suis dans le toit du père dans l'contrôle
j'veux pas faire du mic de ton courage arrivée
c'est pour mes bandits et au grise, c'est nous

<REFRAIN>

le contrat part en sous d'la bouche
c'est la bièche en couille à charlouser le sang
j'ai des cents qu'on est des construits dans la teste
l'amour est des tours de s'conseile de rester comme un coup d'amour
j'ai perdu le biff et j'suis comme le comme de toi
les plus grandes déprimes sont des sous d'la base
tu veux pas durer de la rechange des textes
demain j'reste dans l'combat d'arrache de mes jambes dans l'cou d'un bout d'mes p'tits mères
j'me rêve d'aller pour le balade comme si le son mais j'te rappelle en avance et j'suis pas tout proté et mes potes s'en sont des chevoirs
j'ai dû l'espoir, j'ai pas l'trou d'amour et l'amis
avant qu'tu connais le sous sans concert
ceux qui vont tous les arrives d'humain pour les barrières
c'est force de plus d'ennemi d'faire des mauvais soirs d'avanc

# If no restriction on the generation part 

In [227]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1, device="cpu")

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

maliceS
/γ
Ε
J'ai le plus monte de la bouche et la vie c'est pas d'la femme
J'suis à l'école dans le son d'la route avec une chose qui m'passe d'la mort
J'ai déjà l'plafond d'la ligne qui s'achète dans des tracks, des mots d'air et j'ai pas d'marondule
J'ai pu plein de sommets qu'on parle de merde
Tout s'aime un peu d'oreille
/ε
Γ
C'est pas la maille avant que l'époque de colon dans l'porc
On parle de l'air
Et j'conseille la confiance, tu veux pas rentrer en l'air
J'me rends les miens j'ai l'sourire quand j'viens peut être un peu d'la street
Le prémis d'appart on a pas d'temps le reste
J'suis dans ma tête et le drame de trop en place
J'vois le monde qu'il faut qu'les chansons sont bien d'la vie qui m'enterre
On dit qu'j'en ai déjà pas d'vie d'expérieur
On va pas qu'on fait des soirées
J'relève des putes de princesse
Pas d'se rendre en plein d'toutes les montres de la gueule, on est devenu vivre faut qu'tu vois qu'elle m'a peur de voir la vie comme un cash
J'ai mis un complète de mes de

# Teacher forcing < 1 toward the end of training

In [228]:
new_state_dict = {}
for k, v in ckpt["model_state_dict"].items():
    new_key = k.replace("_orig_mod.", "")  # supprime le préfixe "origin."
    new_state_dict[new_key] = v

In [229]:
#ckpt = torch.load("..\Models\LSTM_model_not_full_forced.pt", map_location="cpu")
#torch.compile(model)
model.load_state_dict(new_state_dict)

<All keys matched successfully>

Forbidden to generate

In [230]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1, device="cpu")

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoi faire les potes dans les rimes mai

s quand j'prends la drogue, pas d'taffer les baillets en moi comme la départ
je me perds dans le plus allez pas de ton parceau de cash
j'suis pas un film du bon cancesse, j'suis bien là-bas d'la merde
c'est mon frère, j'vais pas t'faire mieux de la mort
ouais j'vois qu'ce soir c'est la perte, de l'invente de la main
j'préfère ce qu'on fait du coup, j'écris le change
j'ai l'agot d'distance, on s'batraite avec le jeu
combien d'paradis d'accoude en peine
et j'connais la routine qui font des meilleurs pour baiser des mauvais de chantements d'action
j'ai pas d'décider d'la défile et ta silence
mais j'suis des frères dans le soleil de la folie
pour les mains d'projets sur les albums sans aller dans mon temps qui dit qu'j'suis pas là
j'suis pas d'la famille à l'autre autre bullet de la pagaille
j'suis dans la porte de l'oreille pour le bété d'croire qu'on parle pas
j'suis pas l'impression, j'suis dans l'histoire t'as trop d'temps, c'est pas le bon dieu
j'suis dans l'sourire du bruit du magie,

# If no restriction on the generation part 

In [231]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1, device="cpu")

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

malice
J'ai déconnenté dans les couches, les textes s'en fait des têtes de la sentière
J'ai pas d'marre de ma bande, on a toujours la la base de l'ancienne
J'suis pas là pour la mort et l'enfer, j'ai tout dit: j'suis pas d'mes parents de chaque meilleur
J'ai des rappeurs me poussent, tu vis on est fraitique
Avec les gosses qui peuvent qu'on s'engue la chance et le sang
J'ai grandi dans l'coeur à belle, tu veux qu'j'm'en bats
Qu'est-ce qu'on m'a dit: je n'me vois pas de trois textes j'réponds la concurrence
J'ai dit qu'ça s'en plaît l'été honnête comme les couplets qui s'dégagnent
Mais j'ai pas l'temps d'mes potes de sa fin
J'viens d'chanter la poisse, les basons mais pas de maillans
Et c'est le temps qui prend le connaissant, peux pas t'faire pas la famille
A le nord mais j'ai des sols d'attendre
J'suis dans le monde pour les gars qu'j'ai pas d'pensées d'artiste
J'te la fais pas d'crise et ça m'voit plein d'mes rimes dans l'appriste
J'ai plus d'eux comme les prières pour m'entendre d'l